In [1]:
import os
import shutil
import numpy as np
import zarr
from aicsimageio import AICSImage, writers

from matplotlib.pyplot import imshow

In [ ]:
# utility functions
# define file path
def dataPath(
    dirPath: str = "/",
    fileName: str = None,
    fileID: int = 0,
    outputMode: int = 0  # output options: 0: file path, 1:splitted path, 2: file list
):

    fileList = os.listdir(dirPath)
    if fileName is None:
        fileName = fileList[fileID]
    fileName, filExt = os.path.splitext(fileName)
    filePath = os.path.join(dirPath, fileName + filExt)

    assert outputMode in [0, 1, 2], "outputMode must be 0 or 1"
    match outputMode:
        case 0:  # return file path
            return filePath
        case 1:  # return file path and file extension
            return dirPath, fileName, filExt
        case 2:  # return names in a file directory
            return fileList
        case _:
            raise ValueError(
                "outputMode must be 0-2 (0: file path, 1:splitted path, 2: file list")


# load data into an AICSImage object
def dataLoader(filepath: str = None):
    assert filepath != None, "filepath is None"
    assert os.path.exists(filepath), "file does not exist"
    assert os.path.splitext(filepath)[1] in [
        ".tif", ".tiff", ".czi", ".obf"], "file type is not supported"
    img = AICSImage(filepath)
    return img


# save file into a zarr dircetory
def newZarr(
    pathZarr: str = "/", # the path of the target zarr file
    img: AICSImage = None,
    remove: bool = True
):
    if os.path.splitext(pathZarr)[1] != ".zarr":
        pathZarr = os.path.splitext(pathZarr)[0] + ".zarr"

    assert img != None, "img is None"
    if os.path.exists(pathZarr):
        if remove:
            shutil.rmtree(pathZarr)
        else:
            raise OSError("file exists")
    
    os.mkdir(pathZarr)
    image = img.data
    axes = img.dims.order.lower()
    imgShape = img.shape

    f = zarr.open(pathZarr, mode='w')
    for ii in range(imgShape[1]):
        cTitle = f"C{ii:02d}"
        channel= f.create_group(cTitle)
        for jj in range(imgShape[2]):
            zTitle = f"Z{jj:04d}"
            channel[zTitle] = image[0, ii, jj, :, :]
            channel[zTitle].attrs['resolution'] = (1, 1)
            print(f"channel {ii}, slice {jj} is saved")
        



# convert all files in a directory to zarr
def zarrConverter(
    path: str = "/", # the path of the file directory
    remove: bool = True,
    dataTypes: str = ".tif"
):

    assert os.path.exists(path), "file does not exist"

    fileList = dataPath(path, outputMode=2)
    assert len(fileList) > 0, "file directory is empty"

    filePaths = []
    for file in fileList:
        if file.endswith(dataTypes):
            filePath = dataPath(path, file)
            dirPath, fileName, _ = dataPath(path, file, outputMode=1)
            zarrPath = os.path.join(dirPath, (fileName + '.zarr'))
            filePaths.append(zarrPath)

            img = dataLoader(filePath)
            newZarr(
                pathZarr=zarrPath,
                img=img,
                remove=remove
            )
    assert len(filePaths) > 0, "no files are converted"
    return filePaths